In [12]:
text_lines = open("example1.txt").read().split("\n")

[print(line) for line in text_lines]

next_action = {
    "^": ">",
    ">": "v",
    "v": "<",
    "<": "^"
}

coord_start = [
    (text_lines.index(text_line), text_line.index("^")) 
    for text_line in text_lines if "^" in text_line
][0]
print(f"coord_start: {coord_start}")

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
coord_start: (6, 4)


In [15]:
def replace_char(text, index, char):
    return text[:index] + char + text[index+1:]


replace_char("0123456", 3, "X")

'012X456'

In [63]:
def move(map, coords, debug=False):
    curr_x, curr_y = coords
    direction = map[curr_x][curr_y]
    if direction == "^":
        new_x, new_y = (curr_x - 1, curr_y)
    if direction == ">":
        new_x, new_y = (curr_x, curr_y + 1)
    if direction == "v":
        new_x, new_y = (curr_x + 1, curr_y)
    if direction == "<":
        new_x, new_y = (curr_x, curr_y - 1)
    # DETECT OUT OF MAP
    if new_x < 0 or new_y < 0 or new_x >= len(map) or new_y >= len(map[0]):
        map[curr_x] = replace_char(map[curr_x], curr_y, "X")
        return (-1, -1), map
    # DETECT OBSTACLES
    if (map[new_x][new_y] == "#"):
        if debug: print(f"> Crash at {new_x, new_y}")
        new_action = next_action[direction]
        map[curr_x] = replace_char(map[curr_x], curr_y, new_action)
        return (curr_x, curr_y), map
    # ELSE CONTINUA
    if debug: print(f"> Walking from {curr_x, curr_y} to {new_x, new_y}")
    map[curr_x] = replace_char(map[curr_x], curr_y, "X")
    map[new_x] = replace_char(map[new_x], new_y, direction)
    return (new_x, new_y), map
    

In [66]:
text_lines = open("example1.txt").read().split("\n")
coord_start = [
    (text_lines.index(text_line), text_line.index("^")) 
    for text_line in text_lines if "^" in text_line
][0]

coords = coord_start
map = text_lines.copy()
while (coords != (-1, -1)):
    coords, map = move(map, coords, debug=True)
print(f"Finished!")

> Walking from (6, 4) to (5, 4)
> Walking from (5, 4) to (4, 4)
> Walking from (4, 4) to (3, 4)
> Walking from (3, 4) to (2, 4)
> Walking from (2, 4) to (1, 4)
> Crash at (0, 4)
> Walking from (1, 4) to (1, 5)
> Walking from (1, 5) to (1, 6)
> Walking from (1, 6) to (1, 7)
> Walking from (1, 7) to (1, 8)
> Crash at (1, 9)
> Walking from (1, 8) to (2, 8)
> Walking from (2, 8) to (3, 8)
> Walking from (3, 8) to (4, 8)
> Walking from (4, 8) to (5, 8)
> Walking from (5, 8) to (6, 8)
> Crash at (7, 8)
> Walking from (6, 8) to (6, 7)
> Walking from (6, 7) to (6, 6)
> Walking from (6, 6) to (6, 5)
> Walking from (6, 5) to (6, 4)
> Walking from (6, 4) to (6, 3)
> Walking from (6, 3) to (6, 2)
> Crash at (6, 1)
> Walking from (6, 2) to (5, 2)
> Walking from (5, 2) to (4, 2)
> Crash at (3, 2)
> Walking from (4, 2) to (4, 3)
> Walking from (4, 3) to (4, 4)
> Walking from (4, 4) to (4, 5)
> Walking from (4, 5) to (4, 6)
> Crash at (4, 7)
> Walking from (4, 6) to (5, 6)
> Walking from (5, 6) to (6,

In [67]:
print(f"Final map:")
_ = [print(line) for line in map]

distinct_positions = 0
for line in map:
    distinct_positions += line.count("X")
print(f"Distinct positions: {distinct_positions}")

Final map:
....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
Distinct positions: 41


# Part 1

In [68]:
text_lines = open("example1.txt").read().split("\n")
text_lines = open("input.txt").read().split("\n")

coord_start = [
    (text_lines.index(text_line), text_line.index("^")) 
    for text_line in text_lines if "^" in text_line
][0]
coords = coord_start
map = text_lines.copy()
while (coords != (-1, -1)):
    coords, map = move(map, coords)
print(f"Finished!")

distinct_positions = 0
for line in map:
    distinct_positions += line.count("X")
print(f"Distinct positions: {distinct_positions}")

Finished!
Distinct positions: 5095


<br>

# Part 2

In [145]:
import time

def move(map: list, coords: tuple, repeated: int, debug=False):
    curr_x, curr_y = coords
    direction = map[curr_x][curr_y]
    if debug: print(f"> Curr {curr_x, curr_y}: {direction}")
    if direction == "^":
        new_x, new_y = (curr_x - 1, curr_y)
    if direction == ">":
        new_x, new_y = (curr_x, curr_y + 1)
    if direction == "v":
        new_x, new_y = (curr_x + 1, curr_y)
    if direction == "<":
        new_x, new_y = (curr_x, curr_y - 1)
    # DETECT OUT OF MAP
    if new_x < 0 or new_y < 0 or new_x >= len(map) or new_y >= len(map[0]):
        map[curr_x] = replace_char(map[curr_x], curr_y, "X")
        return (-1, -1), map, repeated
    # DETECT LOOP
    if map[new_x][new_y] == "X":
        repeated += 1
    # DETECT OBSTACLES
    if (map[new_x][new_y] == "#"):
        if debug: print(f"> Crash at {new_x, new_y}")
        new_action = next_action[direction]
        map[curr_x] = replace_char(map[curr_x], curr_y, new_action)
        return (curr_x, curr_y), map, repeated
    # ELSE CONTINUE
    if debug: print(f"> Walking from {curr_x, curr_y} to {new_x, new_y}")
    map[curr_x] = replace_char(map[curr_x], curr_y, "X")
    map[new_x] = replace_char(map[new_x], new_y, direction)
    return (new_x, new_y), map, repeated
    

In [146]:
text_lines = open("example1.txt").read().split("\n")
# text_lines = open("input.txt").read().split("\n")

text_lines[6] = replace_char(text_lines[6], 3, "#")

coord_start = [
    (text_lines.index(text_line), text_line.index("^")) 
    for text_line in text_lines if "^" in text_line
][0]
repeated = 0
coords = coord_start
map = text_lines.copy()
while (coords != (-1, -1)):
    coords, map, repeated = move(map, coords, repeated)
    if repeated > len(map) * len(map[0]):
        print(f"Loop detected!")
        coords = (-2, -2)
        break
print(f"Finished! Result: { coords }")

distinct_positions = 0
for line in map:
    distinct_positions += line.count("X")
print(f"Distinct positions: {distinct_positions}")

Loop detected!
Finished! Result: (-2, -2)
Distinct positions: 17


In [147]:
text_lines = open("example1.txt").read().split("\n")
_ = [print(line) for line in text_lines]

def calculate_possible_coords(map: list):
    possible_coords = []
    for i, line in enumerate(text_lines):
        for j, char in enumerate(line):
            if char not in ["#", "^"]:
                possible_coords.append((i, j))
    return possible_coords

possible_coords = calculate_possible_coords(text_lines)
print(f"Possible coords: {possible_coords}")

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
Possible coords: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 2), (6, 3), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 9), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 7), (9, 8), (9, 9)]


In [161]:
text_lines = open("example1.txt").read().split("\n")
text_lines = open("input.txt").read().split("\n")

coord_start = [
    (i, line.index("^")) 
    for i, line in enumerate(text_lines) if "^" in line
][0]

# possible_coords = [(0, 0), (1, 1), (6, 3)]
possible_coords = calculate_possible_coords(text_lines)
loops = 0
for possible_coord in possible_coords:
    map = text_lines.copy()
    possible_x, possible_y = possible_coord
    map[possible_x] = replace_char(map[possible_x], possible_y, "#")
    # _ = [print(line) for line in map]

    repeated = 0
    coords = coord_start
    while (coords != (-1, -1)):
        coords, map, repeated = move(map, coords, repeated, debug=False)
        if repeated > len(map) * len(map[0]):
            print(f"Loop detected!")
            loops += 1
            coords = (-2, -2)
            break

print(f"Loops detected: {loops}")

Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detected!
Loop detec

> Loops detected: 1933 (1min 50sec)